<a href="https://colab.research.google.com/github/Fridman77/LFPC-Labs/blob/main/Lab5LFPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
from nltk import Tree
from collections import defaultdict
import pandas as pd

In [77]:
Vn = ['S', 'B', 'D', 'A']
Vt = ['a', 'b', 'c', 'd']
P = ['S->dB', 'B->a', 'B->aA', 'A->D', 'A->DcA', 'D->bB', 'A->c']

In [78]:
def print_grammar(states):
    print('\nGIVEN GRAMMAR', end='\n')
    print('Vn = {', str(Vn).replace('[', '').replace(']', '').replace("'", ''), '}')
    print('Vt = {', str(Vt).replace('[', '').replace(']', '').replace("'", ''), '}')
    print('P = {')
    for left_part, right_part in states.items():
        for productions in right_part:
            right = ''
            for character in productions:
                right += character
            print('\t' + left_part + ' -> ' + right)
    print('}')

In [79]:
def check_grammar():

    for k in P:
        if k[0] not in states.keys():
            states[k[0]] = []
        for symbol in k:
            if symbol == '$':
                print('\nThe grammar is not suitable for this parsing algorithm.')
                quit()
    print('\nThe grammar is suitable for this parsing algorithm.')
    print()

In [80]:
def parse_grammar():
    for k in P:
        symbols = []
        if k[0] not in states.keys():
            states[k[0]] = []
        for symbol in k:
            if symbol != '-' and symbol != '>':
                symbols.append(symbol)
        states[symbols.pop(0)].append(symbols)

In [81]:
def split_prod(P):
    for k in Vn:
        prod[k] = []
    for el in P:
        prod[el[0]].append(el[(el.index("->") + 2):])

In [82]:
def get_first_last(left, left1, pos, dict):
    for right in states[left1]:
        if right[pos] not in dict[left]:
            dict[left].append(right[pos])
            if right[pos] in Vn:
                get_first_last(left, right[pos], pos, dict)

In [83]:
def first_last():
    for N in Vn:
        first[N] = []
        last[N] = []
        get_first_last(N, N, 0, first)
        get_first_last(N, N, -1, last)

In [84]:
def print_first_last():
    dd = defaultdict(list)
    for d in (first, last):
        for key, value in d.items():
            dd[key].append(value)
    df = pd.DataFrame([[key] + list(value) for key, value in dd.items()], columns=['VN', 'FIRST', 'LAST'])
    print(df.to_string(index=False))

In [85]:
def get_equal_rule1(production, count): # for productions with 2 or more symbols in the right part
    simple_precedence_matrix[production[count]][production[count + 1]].append('=')

In [86]:
def get_less_rule2(production, count): # productions where Vt||Vn followed by Vn
    if production[count + 1] in Vn:
        for symbol in first[production[count + 1]]:
            simple_precedence_matrix[production[count]][symbol].append('<')

In [87]:
def get_greater_rule3(production, count): # for productions where Vn followed by Vt or Vn0 followed by Vn1
    if production[count] in Vn and production[count + 1] in Vt:
        for symbol in last[production[count]]:
            simple_precedence_matrix[symbol][production[count + 1]].append('>')
    elif production[count] in Vn and production[count + 1] in Vn:
        for symbol in last[production[count]]:
            for symb in first[production[count + 1]]:
                if symb in Vt:
                    simple_precedence_matrix[symbol][symb].append('>')

In [88]:
def start_matrix(array):
    for k in array:
        simple_precedence_matrix[k] = {}
        for element in array:
            simple_precedence_matrix[k][element] = []
            if k == '$' and element != '$':
                simple_precedence_matrix['$'][element] = ['<']
        if k != '$':
            simple_precedence_matrix[k]['$'] = ['>']

In [89]:
def finish_matrix(states):
    start_matrix(symbols_total)
    for left, right in states.items():
        for production in right:
            if len(production) > 1:
                count = 0
                while count < len(production) - 1:
                    get_equal_rule1(production, count)
                    get_less_rule2(production, count)
                    get_greater_rule3(production, count)
                    count += 1

In [90]:
def parse_string(string_input):
    accepted, not_accepted = False, False
    input0 = "$" + string_input + "$"
    w = ''
    for sb in range(len(input0) - 1):
        w += input0[sb] + spm.loc[input0[sb], input0[sb + 1]]
    w += "$"
    input0 = w
    print(input0)
    while not (accepted or not_accepted):
        if input0 == "$<S>$":
            accepted = True
            break
        end = input0.index('>')
        start = input0.rindex('<', 0, end)
        deriv = input0[start+1:end]
        if '=' in deriv:
            deriv = deriv.replace('=', '')
        ant = input0[start-1]
        post = input0[end+1]
        n, not_accepted = check_input(deriv, ant, post)
        if not_accepted:
            break
        input0 = input0[:start] + spm.loc[ant, n] + n + spm.loc[n, post] + input0[end + 1:]
        print(input0)
    if accepted:
        print('Word accepted')
    elif not_accepted:
        print('Word not accepted')

In [91]:
def check_input(deriv, ant, post):
    list = []
    for k, v in prod.items():
        if deriv in v:
            if spm.loc[ant, k] != '' and spm.loc[k, post] != '':
                list.append(k)
    if len(list) == 0:
        return None
    elif len(list) == 1:
        return list[0], False
    else:
        for el in list:
            if (spm.loc[ant, el] == '=' and spm.loc[el, post] == '=') or spm.loc[el, post] == '=' or spm.loc[ant, el] == '=':
                return el, False
        return list[0]


In [92]:
symbols_total = Vn + Vt
symbols_total.append('$')

prod = {}
states = {}
first = {}
last = {}
simple_precedence_matrix = {}

parse_grammar()
split_prod(P)
print_grammar(states)
check_grammar()
first_last()
print_first_last()
finish_matrix(states)


GIVEN GRAMMAR
Vn = { S, B, D, A }
Vt = { a, b, c, d }
P = {
	S -> dB
	B -> a
	B -> aA
	A -> D
	A -> DcA
	A -> c
	D -> bB
}

The grammar is suitable for this parsing algorithm.

VN     FIRST            LAST
 S       [d] [B, a, A, D, c]
 B       [a] [a, A, D, B, c]
 D       [b] [B, a, A, D, c]
 A [D, b, c] [D, B, a, A, c]


In [74]:
data = [vv for v in simple_precedence_matrix.values() for vv in v.values()]
d = []
for el in data:
    d.append(str(el).replace('[]', '').replace("['", '').replace("']", ''))
d = [d[x:x + len(symbols_total)] for x in range(0, len(d), len(symbols_total))]
spm = pd.DataFrame(d, index=symbols_total, columns=symbols_total)
print('\nCOMPUTE SIMPLE PRECEDENCE MATRIX', end='\n')
print(spm)


COMPUTE SIMPLE PRECEDENCE MATRIX
   S  B  D  A  a  b       c  d  $
S                               >
B                         >     >
D                    =', '>     >
A                         >     >
a        <  =     <  <', '>     >
b     =        <                >
c        <  =     <  >', '<     >
d     =        <                >
$  <  <  <  <  <  <       <  <   


In [93]:
string_input = 'dabacba'

print('\nPARSE WORD:', string_input, end='\n')
parse_string(string_input)


PARSE WORD: dabacba
$<d<a<b<a<', '>c<b<a>$


TypeError: ignored